## N-BEATS (Darts) ? 1/7/30 kunlik forecast


In [ ]:
print('hi')

hi


In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

In [17]:
# Ma'lumotlarni yuklash
p = Path('datasets/usd_rates_ready.csv')
df = pd.read_csv(p)

df.columns = [c.lower() for c in df.columns]
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Target = rate, date ishlatilmaydi, qolgan 21 ta ustun feature sifatida
feature_cols = [c for c in df.columns if c not in ['date', 'rate']]

# Outlierlarni IQR bilan yumshatish (faqat targetda)
def clip_iqr(s, k=1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - k * iqr, q3 + k * iqr
    return s.clip(lower, upper)

df['rate'] = clip_iqr(df['rate'])

# Vaqt bo'yicha train/val/test (70/15/15)
n = len(df)
train_end = int(n * 0.70)
val_end = int(n * 0.85)
train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

# Bashorat ufqlari
horizons = [1, 7, 30]

try:
    from darts import TimeSeries
    from darts.models import NBEATSModel
    from darts.dataprocessing.transformers import Scaler
except ImportError:
    raise SystemExit("Darts o'rnatilmagan: pip install darts[u]")

series = TimeSeries.from_dataframe(df, time_col='date', value_cols='rate')
covariates = TimeSeries.from_dataframe(df, time_col='date', value_cols=feature_cols)

scaler_y = Scaler()
scaler_x = Scaler()

series_s = scaler_y.fit_transform(series)
cov_s = scaler_x.fit_transform(covariates)

train, rest = series_s.split_before(0.70)
val, test = rest.split_before(0.50)

cov_train, cov_rest = cov_s.split_before(0.70)
cov_val, cov_test = cov_rest.split_before(0.50)

model = NBEATSModel(
    input_chunk_length=50,
    output_chunk_length=10,
    n_epochs=30,
    random_state=0,
    pl_trainer_kwargs={
    "accelerator": "gpu",
    "devices": "auto",  # yoki 1, 2
}
)
model.fit(train, past_covariates=cov_train, val_series=val, val_past_covariates=cov_val)

val_pred = model.predict(n=len(val), series=train, past_covariates=cov_train.concatenate(cov_val))
test_pred = model.predict(n=len(test), series=train.concatenate(val), past_covariates=cov_train.concatenate(cov_val).concatenate(cov_test))

mae_val = mean_absolute_error(val.values(), val_pred.values())
rmse_val = root_mean_squared_error(val.values(), val_pred.values())
r2_score_val = r2_score(val.values(), val_pred.values())


mae_test = mean_absolute_error(test.values(), test_pred.values())
rmse_test = root_mean_squared_error(test.values(), test_pred.values())
r2_score_test = root_mean_squared_error(test.values(), test_pred.values())

print(f"Vali MAE: {mae_val:.2f},  RMSE: {rmse_val:.4f},  R2 Score: {r2_score_val:.4f}")
print(f"Test MAE: {mae_test:.2f}, RMSE: {rmse_test:.4f}, R2 Score: {r2_score_test:.4f}")


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 14.7 M | train
-------------------------------------------------------------
14.7 M    Trainable params
7.9 K     Non-trainable params
14.7 M    Total params
58.767    Total estimated model params size (MB)
396       Modules in train mode
0         Modules in eval mode


Epoch 29: 100%|██████████| 55/55 [00:03<00:00, 14.10it/s, train_loss=0.000205, val_loss=0.00176] 

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 55/55 [00:03<00:00, 14.09it/s, train_loss=0.000205, val_loss=0.00176]


`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.74it/s]

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s]
Vali MAE: 0.03,  RMSE: 0.0409,  R2 Score: -0.4376
Test MAE: 0.04, RMSE: 0.0515, R2 Score: 0.0515


In [ ]:
# 1/7/30 forecast: covariates oxirgi qiymati bilan uzaytiriladi
horizon_forecasts = {}
for h in horizons:
    future_dates = pd.date_range(df['date'].max() + pd.Timedelta(days=1), periods=h, freq='D')
    last_cov = cov_s.values()[-1]
    future_cov_vals = np.repeat(last_cov[np.newaxis, :], h, axis=0)
    future_cov = TimeSeries.from_times_and_values(future_dates, future_cov_vals)
    full_cov = cov_s.concatenate(future_cov)
    fc = model.predict(n=h, series=series_s, past_covariates=full_cov)
    fc_inv = scaler_y.inverse_transform(fc)
    horizon_forecasts[f'h_{h}'] = fc_inv.to_dataframe()

horizon_forecasts['h_30'].head(10)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 33.13it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 37.89it/s]

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 14.82it/s]


,rate
date,
2025-12-10,12131.446263
2025-12-11,12085.788230
2025-12-12,12130.228725
2025-12-13,12223.618197
2025-12-14,12089.896275
2025-12-15,12122.368384
2025-12-16,12177.249612
2025-12-17,12115.496128
2025-12-18,12135.593793


In [ ]:
# Bashoratni vizualizatsiya qilish
import matplotlib.pyplot as plt

def plot_forecast(h_key='h_7', title='7 kunlik forecast'):  # h_key: 'h_1', 'h_7', 'h_30'
    fc_df = horizon_forecasts[h_key].sort_values('ds').copy()
    fc_df['delta'] = fc_df['yhat'].diff()

    plt.figure(figsize=(8, 4))
    plt.plot(fc_df['ds'], fc_df['yhat'], label='yhat (bashorat)', color='C0', alpha=0.5)
    plt.fill_between(fc_df['ds'], fc_df['yhat_lower'], fc_df['yhat_upper'], color='C0', alpha=0.15, label='ishonch oralig\'i')

    inc_mask = fc_df['delta'] >= 0
    colors = np.where(inc_mask, 'green', 'red')
    plt.scatter(fc_df['ds'], fc_df['yhat'], c=colors, edgecolor='none', s=40, label='Ko\'tarilish (yashil) / tushish (qizil)')

    plt.title(title)
    plt.xlabel('Sana')
    plt.ylabel('Kurs')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Default: 7 kunlik forecast
plot_forecast('h_30', '30 kunlik forecast')
